In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load Data

In [ ]:
df = pd.read_csv("../input/retailtransactiondata/Retail_Data_Transactions.csv")
df

# Check Missing & Null Data

In [ ]:
data_missing_value = df.isnull().sum().reset_index()
data_missing_value

Note : Nothing Missing Data 

In [ ]:
df.duplicated().sum()

Note : 6 Data Duplicated

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.duplicated().sum()

Note : Nothing Duplicated Data Anymore

# Recency

In [ ]:
df['trans_date']=pd.to_datetime(df['trans_date'])

In [ ]:
last_date=df['trans_date'].max()
last_date

In [ ]:
recency=df.groupby('customer_id').agg({'trans_date': 'max'}).reset_index()
recency.columns=['customer_id','last_trans']
recency['recency']=last_date-recency['last_trans']
recency=recency.drop(['last_trans'],axis=1)

recency['recency'] = pd.DataFrame(recency['recency'].astype('timedelta64[D]'))

recency

# Frequency

In [ ]:
frequency=df.groupby('customer_id').agg({'tran_amount':'count'}).reset_index()
frequency.columns=['customer_id','frequency']
frequency

# Monetary

In [ ]:
monetary=df.groupby('customer_id').agg({'tran_amount': 'sum'}).reset_index()
monetary.columns=['customer_id','monetary']
monetary

# RFM

In [ ]:
RFM=recency.merge(frequency,
                 on='customer_id')
RFM=RFM.merge(monetary,
             on='customer_id')
RFM

# Standarization

In [ ]:
X=RFM[['recency','frequency','monetary']]

from sklearn.preprocessing import StandardScaler

X_std = StandardScaler().fit_transform(X)
RFM_std = pd.DataFrame(data = X_std, columns = ['recency','frequency','monetary'])
RFM_std.describe()

# K Means

In [ ]:
from sklearn.cluster import KMeans
inertia = []

for i in range(1, 11):
  kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
  kmeans.fit(RFM_std.values)
  inertia.append(kmeans.inertia_)

plt.figure(figsize=(12, 8))
plt.plot(inertia)

Note :there is 4 cluster because formed 3 corner

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(RFM_std.values)

In [ ]:
RFM_std['cluster']= kmeans.labels_

In [ ]:
melted_RFM=pd.melt(RFM_std.reset_index(),
                  id_vars=['cluster'],
                   value_vars=['recency','monetary','frequency'],
                   var_name='Features',
                   value_name='Value'
                  )

sns.lineplot('Features','Value',hue='cluster',data=melted_RFM)
plt.legend()

# Clustering Analysis

First Cluster (Label "2"): Visitors who have shopped but did not come back next time <br>
**Loyalty: NO** <br>
Highest Recency: the type of customer who has not visited for a long time <br>
Monetary Low: Spend money is low <br>
Low Frequency: Rarely shop at the marketplace<br>

Second Cluster (Label "0"): Type of low-class visitor, if the visitor does not come back, it will **potentially become the first cluster** <br>
**Loyalty: LOW** <br>
Recency (time of last visit) is relatively the same as the third & fourth types <br>
Low monetary: low spend money <br>
Low frequency: rarely shop at the marketplace <br>

Third Cluster (Label "3"): Types of visitors to the middle class <br>
**Loyalty: MEDIUM** <br>
Recency (time of last visit) is relatively the same as the third & fourth types <br>
Medium monetary: intermediate spend money <br>
Medium frequency: shopping regularly at the marketplace <br>

Fourth Cluster (Label "1"): Types of high-class visitors <br>
**Loyalty: HIGHT** <br>
Recency (time of last visit) is relatively the same as the third & fourth types <br>
High monetary: the highest / highest spend money <br>
High frequency: shopping very regularly in the marketplace 